### What's [AWS SDK for pandas](https://aws-sdk-pandas.readthedocs.io/en/stable/about.html)?

An AWS Professional Service open source python initiative that extends the power of the pandas library to AWS, connecting DataFrames and AWS data & analytics services.

Easy integration with Athena, Glue, Redshift, Timestream, OpenSearch, Neptune, QuickSight, Chime, CloudWatchLogs, DynamoDB, EMR, SecretManager, PostgreSQL, MySQL, SQLServer and S3 (Parquet, CSV, JSON and EXCEL).

Built on top of other open-source projects like Pandas, Apache Arrow and Boto3, it offers abstracted functions to execute your usual ETL tasks like load/unloading data from Data Lakes, Data Warehouses and Databases, even at scale.

In [2]:
#Importing required libraries
import awswrangler as wr
import yfinance as yf
import boto3
import pandas as pd
import datetime as dt
from datetime import date, timedelta

In [7]:
#Setting datetime variables
today = dt.datetime.now()                #Current day of the month
start_date = today - timedelta(days=365) #1 year back from the current day
end_date = today

In [6]:
#Creating S3 bucket using AWS CLI
!aws s3api create-bucket --bucket aws-sdk-pandas72023 --region us-east-2 --create-bucket-configuration LocationConstraint=us-east-2

{
    "Location": "http://aws-sdk-pandas72023.s3.amazonaws.com/"
}


In [5]:
#Setting the variable for the bucket name
bucket = 'aws-sdk-pandas72023'
path = f"s3://{bucket}/dataset/"

#### AWS S3 integration

##### Helper functions

In [3]:
def get_data_from_api(ticker:str, start_date:str, end_date:str) -> pd.DataFrame:
    """
    Parameters:
    ----------
    tickers(list): List of tickers to be downloaded
    start_date(str): Start date of the data
    end_date(str): End date of the data
    """

    #Downloading data from yahoo finance api
    #If ticker is not available, it will be skipped
    #If ticker is available, it will be downloaded and stored in a dataframe
    #The dataframe will be returned

    try:
        ticker = yf.download(tickers=ticker, start=start_date, end=end_date)
    except:
        pass

    return ticker

def write_data_to_bucket(file_name:str, mode:str):
    """
    Parameters:
    ----------
    mode(str): Available write modes are 'append', 'overwrite' and 'overwrite_partitions'
    """

    path = f"s3://{bucket}/raw-data/{file_name}"
    #Sending dataframe of corresponding ticker to bucket
    wr.s3.to_csv(
        df=df,
        path=path,
        index=True,
        dataset=True,
        mode=mode
    )

def read_csv_from_bucket(folder_name:str) -> pd.DataFrame:

    df = wr.s3.read_csv(path = f"s3://{bucket}/raw-data/{folder_name}/",
                        path_suffix = ".csv"
)
    return df



##### Writing data to S3 bucket

Downloading a single dataframe from API and loading to S3 bucket

In [9]:
ticker = 'NVDA'

#Getting data fro API corresponding to the ticker symbol `NVIDIA Corporation`
df = get_data_from_api(ticker,start_date,end_date)

#Writing data to the bucket
write_data_to_bucket(ticker,'overwrite')

[*********************100%***********************]  1 of 1 completed


Downloading a multiple dataframes from API and loading to S3 bucket

In [23]:
tickers = ['C', 'CAT', 'CL']

for ticker in tickers:
    #Downloading data from API
    df = get_data_from_api(ticker, start_date, end_date)
    #Writing data to the bucket
    write_data_to_bucket(ticker,'overwrite')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


##### Reading data from bucket

In [24]:
read_csv_from_bucket('CAT')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-07-21,178.419998,181.070007,176.479996,180.990005,177.090622,2214500
1,2022-07-22,181.490005,182.800003,177.339996,178.619995,174.771698,1961500
2,2022-07-25,179.169998,182.600006,177.770004,181.809998,177.892975,1845600
3,2022-07-26,181.160004,183.649994,180.580002,181.229996,177.325470,1767700
4,2022-07-27,182.919998,186.179993,180.320007,185.250000,181.258881,1856500
...,...,...,...,...,...,...,...
246,2023-07-14,256.480011,256.480011,252.910004,255.619995,254.360352,1936800
247,2023-07-17,254.270004,258.850006,252.009995,257.459991,256.191284,2674500
248,2023-07-18,257.630005,264.160004,256.929993,263.809998,262.509979,3833400
249,2023-07-19,260.140015,262.920013,259.700012,262.750000,262.750000,2771900


In [31]:
wr.s3.does_object_exist(f's3://{bucket}/raw-data/CAT/3fc6c37a1c344286b83ecb93e83248a2.csv')

True